### Linear Regression
Given a set of observations $x^{(i)}$ and $y^{(i)}$, the objective is to find a function $f: X \rightarrow Y$ of the form $f(x)=a_{0} + a_{1}x$ 
that minimizes a loss function $l: f(x^{(i)}), y^{(i)} \rightarrow L $. 

In the vectorized form of the function where $a = \begin{bmatrix} a_0 \\ a_1 \end{bmatrix}$ and $X = \begin{bmatrix} 1 & x \end{bmatrix}$ $$f(x)=X \cdot a$$

In this case the loss function is the quadratic loss function $$\begin{equation} 
L(a) = \sum_{i=1}^{N} (y^{(i)}-f(x^{(i)}))^2 = \sum_{i=1}^{N} (y^{(i)} - a \cdot X^{(i)})^2 \end{equation}$$ 

After the differentiation of the function our purpose is to find the zero of the derivatives
$$ \begin{align*} 
\frac{dL}{da} &= - 2 \sum_{i=1}^{N}(y^{(i)} - a \cdot X^{(i)} ) \cdot X^{T (i)} \\
\sum_{i=1}^{N} y^{(i)} \cdot X^{T_{(i)}} &=  \sum_{i=1}^{N} a \cdot X^{(i)} \cdot X^{T_{(i)}} \\
X^T \cdot y &= a \cdot X \cdot X^T \\
a &= X^T \cdot y \cdot (X \cdot X^T)^{-1} 
\end{align*}$$


In [1]:
# pip install numpy 
import numpy as np
"""
Perform linear regression using the conventional way using linear algebra.
Args:
- X (list): Input feature values.
- Y (list): Observed output values.
Returns:
- list: Parameter vector 'a' for the linear regression model.
"""
def linear_regression(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    
    X = np.column_stack((np.ones_like(X), X))
    
    X_t = np.transpose(X)
    XtX_i = np.linalg.inv(np.dot(X_t, X))
    a = np.dot(np.dot(XtX_i, X_t), Y)
    
    return a

In [3]:
X = [1, 2, 3, 4, 5]
Y = [2, 4.05, 6, 8, 10.2]

parameters = linear_regression(X, Y)
print("Parameter vector a:", parameters)

Parameter vector a: [-0.055  2.035]


### Gradient Descent

Given a set of observations $x^{(i)}$ and $y^{(i)}$, the objective is to find a function $f: X \rightarrow Y$ of the form $f(x) = a_0 + a_1x$ that minimizes a loss function $L$. The loss function quantifies the difference between the predicted values $f(x^{(i)})$ and the actual values $y^{(i)}$.

The function can be expressed in a vectorized form where $a = \begin{bmatrix} a_0 \\ a_1 \end{bmatrix}$ and $X = \begin{bmatrix} 1 & x \end{bmatrix}$ for each observation, leading to a prediction model $f(x) = Xa$.

The chosen loss function is the quadratic loss function defined as:
$$
L(a) = \sum_{i=1}^{N} \left(y^{(i)} - f(x^{(i)})\right)^2 = \sum_{i=1}^{N} \left(y^{(i)} - X^{(i)}a\right)^2
$$

Gradient descent aims to minimize $L(a)$ by iteratively adjusting $a$ in the direction that most rapidly decreases $L(a)$. This is done by computing the gradient of $L(a)$ with respect to $a$ and updating $a$ using the formula:
$$
a_{\text{new}} = a_{\text{old}} - \alpha \nabla L(a)
$$
where $\alpha$ is the learning rate, a scalar that determines the step size during each iteration.

The gradient $\nabla L(a)$ is calculated as:
$$
\nabla L(a) = -2 \sum_{i=1}^{N} \left(y^{(i)} - X^{(i)}a \right)X^{(i)} 

$$

By iteratively applying this update rule, gradient descent seeks to find $a$ that minimizes $L(a)$, effectively training the linear regression model.


In [4]:
def predict(a, X):
    """
    Predicts the output using a linear regression model.

    Parameters:
    - a (numpy array): Coefficients for the linear regression model (a_0, a_1).
    - X (list): Input features.

    Returns:
    - numpy array: Predicted values.
    """
    X = np.array(X)
    X = np.column_stack((np.ones_like(X), X))

    return np.dot(a,X.T)

In [5]:
def error(a, X, Y):
    Y = np.array(Y)
    return (Y - predict(a, X))

def quadratic_loss(a, X, Y):
    Y = np.array(Y)
    return (Y - predict(a, X)) ** 2
    

In [6]:
def gradient(a, X, Y):
    X_t = np.column_stack((np.ones_like(X), np.array(X))) 
        
    dL_da = - 2 * np.dot(error(a, X, Y), X_t)

    return dL_da

In [7]:
def update_params(a, X, Y, alpha):
    a = a - alpha * gradient(a, X, Y)

    return a

In [8]:
def train(a, X, Y, alpha, epochs=None, verbose=100, min_increase=1e-8):
    """
    Trains the linear regression model using gradient descent.

    Args:
        a (numpy array): Initial parameters.
        X (list): Input features.
        Y (list): Observed values.
        alpha (float): Learning rate.
        epochs (int): Number of iterations to run the gradient descent.
        verbose (int): Number of times to print loss
        min_increase (float): Minimum decrease in error to stop training
    Returns:
        numpy array: Optimized parameters.
    """
    if epochs is None:
        e = 0
        increase = 1e100
        while increase > min_increase:
            error_before = np.sum(quadratic_loss(a, X, Y))
            a = update_params(a, X, Y, alpha)
            error_after = np.sum(quadratic_loss(a, X, Y))
            if e % verbose == 0:
                average_loss = np.sum(quadratic_loss(a, X, Y)) / len(Y)
                print(f"Average loss = {average_loss}")    
            increase = error_before - error_after
            e += 1
        return a
    for e in range(epochs):
        a = update_params(a, X, Y, alpha)
        if e % verbose == 0:
            average_loss = np.sum(quadratic_loss(a, X, Y)) / len(Y)
            print(f"Average loss = {average_loss}")
    return a

In [9]:
a = np.array([0, 1])
X = [1, 2, 3, 4, 5]
Y = [2, 4.05, 6, 8, 10.2]

parameters = train(a, X, Y, alpha=0.01, verbose=100)
print("Parameter vector a:", parameters)


Average loss = 0.40432899999999955
Average loss = 0.004135393480373686
Average loss = 0.0035693447036872277
Average loss = 0.0035506392581627446
Parameter vector a: [-0.05444144  2.03484529]
